# Seminar 3: Data Formats and APIs

March 25, 2025
- Vilém Krejcar

______________________________

## What are we going to learn

- What are the most popular data formats and how to work with them
- Databases, SQL and data manipulation
- HTTP requests
- How to scrape websites 

## 1. Data formats

### Data Serialization and Formats
Serialization is the process of converting data structures or objects into a format that can be easily stored and shared. Common data formats in Python include:
- **JSON**: Lightweight and commonly used for APIs.
- **CSV**: Ideal for tabular data, especially in spreadsheets.
- **Excel**: Classic data storage everyone is aware of.

#### JSON

In [1]:
import json
import pandas as pd
import os
    
data = {
    "name": "Alice",
    "age": 30,
    "is_member": True,
    "hobbies": ["reading", "biking", "coding"]
}

In [2]:
directory = 'Data'
file_path = os.path.join(directory, 'json_sample.json')
os.makedirs(directory, exist_ok=True)

with open(file_path, 'w') as f:
    json.dump(data, f)

In [4]:
with open(file_path, 'r') as f:
    data = pd.read_json(f)

with open(file_path, 'r') as f:
    data = json.load(f)

In [5]:
pd.DataFrame(data)

,name,age,is_member,hobbies
0,Alice,30,True,reading
1,Alice,30,True,biking
2,Alice,30,True,coding


#### CSV

In [25]:
import csv

data = [
    ["Name", "Age", "Occupation"],
    ["Alice", 30, "Engineer"],
    ["Bob", 25, "Designer"],
    ["Charlie", 35, "Teacher"]
]


In [11]:
directory = 'Data'
file_path = os.path.join(directory, 'sample_csv.csv')

os.makedirs(directory, exist_ok=True)

with open(file_path, 'w') as f:
    writer = csv.writer(f)
    writer.writerows(data)

In [14]:
csv_data = []
with open(file_path, 'r') as f:
    data = pd.read_csv(f)

with open(file_path, 'r') as f:
    csv_data = list(csv.reader(f))

In [15]:
csv_data

[['Name', 'Age', 'Occupation'],
 [],
 ['Alice', '30', 'Engineer'],
 [],
 ['Bob', '25', 'Designer'],
 [],
 ['Charlie', '35', 'Teacher'],
 []]

#### Excel

In [23]:
data = {
    "name": "Thomas",
    "age": 30,
    "is_member": True,
    "hobbies": "reading, biking, coding"
}

In [26]:
directory = 'Data'
file_path = os.path.join(directory, 'sample_excel.xlsx')

os.makedirs(directory, exist_ok=True)

df = pd.DataFrame(data)

df.to_excel(file_path, index = False)

## 2. Requests and web scraping

In [27]:
import requests # for making HTTP requests
import pandas as pd 
import time
import re # Regex = Regular Expressions

Time package small things:

In [28]:
%%time
print("Hello, World!")

Hello, World!
CPU times: total: 0 ns
Wall time: 0 ns


In [29]:
t0 = time.time()
time.sleep(1)
t1 = time.time()
print("Time elapsed: ", t1-t0, " seconds")

Time elapsed:  1.0040500164031982  seconds


In [30]:
%%time 
time.sleep(2)

CPU times: total: 0 ns
Wall time: 2 s


In [31]:
import random

In [32]:
%%time
r_time = random.uniform(0.5, 1.2)
print("Sleeping for ", r_time, " seconds")
time.sleep(r_time)

Sleeping for  0.8212864178980297  seconds
CPU times: total: 0 ns
Wall time: 822 ms


#### Task 1: Requesting API

Let us work with data of sreality.cz which we can access via their api. An intuition is that the api is limited for a number of requests (but not verified).

##### 1a. Create a function requesting data from sreality

```python
base_url = 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&locality_region_id=10&per_page60&page={}'.format(i)

r = requests.get(base_url)
d = r.json()
```

0) function should parametrize: 
    * `category_main_cb` - `{'flat':1, 'house':2, 'land':3 }`
    * `category_type_cb` - `{'sell':1,'rent':2}`
    * `locality_region_id` - use 10 as default value
    * `page` parameter
1) use string inputs for `category_main_cb` and `category_type_cb`
2) include `try/except` clause to handle errors
3) function should return JSON data in python types
4) do not forget to sleep each request at least 0.5s

In [50]:
def request_sreality(page, category_main_str, category_type_str, locality_region_id=10):
    """
    Request data from sreality.cz API
    :param page: page number
    :param category_main_str: category of the property
    :param category_type_str: type of the offer
    :param locality_region_id: region id
    :return json: json response
    """
    category_mains = {'flat': 1, 'house': 2, 'land': 3}
    category_types = {'sell': 1, 'rent': 2}
    tempalte_url = 'https://www.sreality.cz/api/cs/v2/estates?category_main_cb={category_main}&category_type_cb={category_type}&locality_region_id={locality_region_id}&per_page60&page={page}'
    request_url = tempalte_url.format(
        category_main = category_mains[category_main_str],
        category_type = category_types[category_type_str],
        locality_region_id = locality_region_id,
        page = page
    )

    response_sleep = random.uniform(0.5, 1.2)
    time.sleep(response_sleep)

    try:
        response = requests.get(request_url)
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(e)
        return None


In [53]:
data = request_sreality(0, 'flat', 'rent')

Inspect the element `d`:

In [57]:
data.keys()

dict_keys(['meta_description', 'result_size', '_embedded', 'filterLabels', 'title', 'filter', '_links', 'locality', 'locality_dativ', 'logged_in', 'per_page', 'category_instrumental', 'page', 'filterLabels2'])

In [61]:
edata = data['_embedded']
edata.keys()

dict_keys(['estates', 'is_saved', 'not_precise_location_count'])

In [67]:
edata['estates'][0].keys()

dict_keys(['labelsReleased', 'has_panorama', 'labels', 'is_auction', 'labelsAll', 'seo', 'exclusively_at_rk', 'category', 'has_floor_plan', '_embedded', 'paid_logo', 'locality', 'has_video', 'advert_images_count', 'new', 'auctionPrice', 'type', 'hash_id', 'attractive_offer', 'price', 'price_czk', '_links', 'rus', 'name', 'region_tip', 'gps', 'has_matterport_url'])

In [68]:
column_names = edata['estates'][0].keys()
df = pd.DataFrame(columns=column_names)

In [69]:
df

,labelsReleased,has_panorama,labels,is_auction,labelsAll,seo,exclusively_at_rk,category,has_floor_plan,_embedded,...,hash_id,attractive_offer,price,price_czk,_links,rus,name,region_tip,gps,has_matterport_url


In [71]:
def convert_sreality_data_to_df(sreality_data):
    return pd.DataFrame(sreality_data['_embedded']['estates'])

In [72]:
df = convert_sreality_data_to_df(data)

In [73]:
df

,labelsReleased,has_panorama,labels,is_auction,labelsAll,seo,exclusively_at_rk,category,has_floor_plan,_embedded,...,hash_id,attractive_offer,price,price_czk,_links,rus,name,region_tip,gps,has_matterport_url
0,"[[after_reconstruction], []]",0,[Po rekonstrukci],False,"[[personal, after_reconstruction, brick], [the...","{'category_main_cb': 1, 'category_sub_cb': 2, ...",0,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,2900562508,0,27000,"{'value_raw': 27000, 'unit': 'za měsíc', 'name...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Pronájem bytu 1+kk 36 m²,3239160,"{'lat': 50.073473912870796, 'lon': 14.46310908...",False
1,"[[balcony, panel, cellar], []]",0,"[Balkon, Panelová, Sklep]",False,"[[personal, balcony, panel, cellar, elevator, ...","{'category_main_cb': 1, 'category_sub_cb': 47,...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,1445737036,0,8000,"{'value_raw': 8000, 'unit': 'za měsíc', 'name'...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Pronájem pokoje 20 m²,0,"{'lat': 50.00034291287079, 'lon': 14.468059087...",False
2,"[[], []]",0,[],False,"[[personal, furnished], [sports, restaurant, s...","{'category_main_cb': 1, 'category_sub_cb': 2, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,635466316,0,15000,"{'value_raw': 15000, 'unit': 'za měsíc', 'name...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Pronájem bytu 1+kk 26 m²,0,"{'lat': 50.059329912870794, 'lon': 14.48421308...",False
3,"[[balcony], [metro]]",0,"[Balkon, Metro 3 min. pěšky]",False,"[[personal, balcony, brick, elevator], [small_...","{'category_main_cb': 1, 'category_sub_cb': 2, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,1319072332,0,15000,"{'value_raw': 15000, 'unit': 'za měsíc', 'name...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Pronájem bytu 1+kk 28 m²,0,"{'lat': 50.06536791287079, 'lon': 14.471080087...",False
4,"[[], [metro, medic]]",0,"[Metro 242 metrů pěšky, Lékař 421 metrů pěšky]",False,"[[personal, brick, not_furnished], [theater, r...","{'category_main_cb': 1, 'category_sub_cb': 7, ...",0,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,168849996,0,26000,"{'value_raw': 26000, 'unit': 'za měsíc', 'name...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Pronájem bytu 3+1 92 m²,0,"{'lat': 50.05818491287079, 'lon': 14.429304087...",False
5,"[[terrace, cellar, parking_lots], []]",0,"[Terasa, Sklep, Parkování]",False,"[[personal, terrace, cellar, parking_lots, par...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,454062668,0,31000,"{'value_raw': 31000, 'unit': 'za měsíc', 'name...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Pronájem bytu 2+kk 62 m²,0,"{'lat': 50.04762591287079, 'lon': 14.292148087...",False
6,"[[new_building, loggia, cellar], []]",0,"[Novostavba, Lodžie, Sklep]",False,"[[new_building, personal, loggia, brick, cella...","{'category_main_cb': 1, 'category_sub_cb': 6, ...",0,1,1,"{'favourite': {'is_favourite': False, '_links'...",...,2281796172,0,36000,"{'value_raw': 36000, 'unit': 'za měsíc', 'name...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,True,Pronájem bytu 3+kk 119 m²,0,"{'lat': 50.1262699128708, 'lon': 14.5648240871...",False
7,"[[cellar], [metro, post_office]]",0,"[Sklep, Metro 3 min. pěšky, Pošta 2 min. pěšky]",False,"[[personal, brick, cellar], [theater, vet, can...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",1,1,0,"{'favourite': {'is_favourite': False, '_links'...",...,2333110860,0,27000,"{'value_raw': 27000, 'unit': 'za měsíc', 'name...",{'dynamicDown': [{'href': 'https://d18-a.sdn.c...,False,Pronájem bytu 2+kk 75 m²,0,"{'lat': 50.05613691287079, 'lon': 14.413430087...",False
8,"[[after_reconstruction, partly_furnished], []]",0,"[Po rekonstrukci, Částečně vybavený]",False,"[[personal, after_reconstruction, brick, eleva...","{'category_main_cb': 1, 'category_sub_cb': 4, ...",1,1,0,"{'favour

### 1b. Create a function converting sreality json data into pandas dataframe

In [54]:
df = pd.DataFrame(data)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [ ]:
def convert_sreality_data_to_df(sreality_data):
    raise NotImplemented

raw = convert_sreality_data_to_df(d)

### 1c. link function `1b` into function `1a`

### 1c. Combining multiple requests into single df

* Function should parametrize:
    * `start_page` and `end_page`
    * request parameters
* construct a list of individual request dfs
* then feed it into `pd.concat` function

In [ ]:
def request_multiply_sreality(start_page, end_page, category_main_str, category_type_str, locality_region_id=10):
    raise NotImplementedError

#### Task 2: Cleaning data

__2a. Filter columns__
* filter only columns: `['locality', 'price', 'name', 'gps','hash_id','exclusively_at_rk']`
* use `.copy()` to avoid `SettingWithCopyWarning` later


Copy selection into new one

### 2b: GPS
* Convert dictionary in `gps` column into two columns - `lat` and `lon`
* use apply function on gps column
* Note apply can return multiple columns

### 2c. Get flat type from name

* Name is always represented by string `Prodej bytu [type of flat] [Area] m^2`
* Try picking third word in string
* Check meaningfulness using `.value_counts()`

In [ ]:
# regular expression to find numbers before m²


In [ ]:
# regular expression to find characters before '+' and behind '+'
def flat_type(x:str):
    # \d+ represents one or more digit(s)
    # \s is a space character
    # \w+ is one or more word character(s)
    # * represents 0 or more
    raise NotImplementedError

In [ ]:
sample_text = 'Prodej bytu 3+kk 71 m²'
print(f"Initial text: {sample_text}")
print("Processed text:")


### 2d. Get the area of a flat from name

* Naive: select the word before last word
* Then try navigating using the index of `'m²'`
* If this also fail, then you will need to use regex - `import re`

In [ ]:
def name_to_area(nm):
    raise NotImplementedError



__Check if the two areas are equal__